![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [33]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [34]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()


In [35]:
# Exploratory data analysis
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [36]:
# Create a dictionary of columns containing ordinal cateforical data 

ordered_data = {
    'enrolled_univeristy': ['no_enrollment', 'Part time course', 'Full time course'],
    'education_level': ['Primary School', 'High School', 'Graduate', ' Masters', 'PhD'],
    'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    'company_size': ["<10", "10–49", "50–99", "100–499", "500–999", "1000–4999", "5000–9999", "10000+"],
    'last_new_job': ['never'] + list(map(str, range(1, 5))) + ['>4']
}

In [37]:
# Create a for loop through the ds_jobs_transformed to change data types

for col in ds_jobs_transformed.columns:
    
    # Columns containing categories with only two factors must be stored as Booleans
    if col in ['relevant_experience', 'job_change']:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('bool')
        
    # Columns containing integers only must be stored as 32-bit integers
    elif ds_jobs_transformed[col].dtype == 'int':
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('int32')
        
    # Columns containing floats must be stored as 16-bit floats
    elif ds_jobs_transformed[col].dtype == 'float':
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('float16')
    
    # Columns containing ordinal categorical data must be stored as ordered categories
    elif col in ordered_data.keys():
        category = pd.CategoricalDtype(ordered_data[col], ordered = True)
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')
    
    # Columns containing nominal categorical data must be stored as the category data type
    else:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')

In [38]:
# Explore the data after the transformation
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  bool    
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  bool    
dtypes: bool(2), category(9

In [39]:
# Check and compare memory usage of the two dataframe:

ds_jobs_mu = ds_jobs.memory_usage(deep = True).sum()
print(f'Total memory usage of the ds_jobs is {ds_jobs_mu} bytes')

ds_jobs_transformed_mu = ds_jobs_transformed.memory_usage(deep = True).sum()
print(f'Total memory usage of the ds_jobs is {ds_jobs_transformed_mu} bytes')

percen = ((ds_jobs_mu - ds_jobs_transformed_mu)/ds_jobs_mu)*100
print(f'After optimization, we achieved a significant reduction in memory usage which is {percen: .2f}%')

Total memory usage of the ds_jobs is 12391705 bytes
Total memory usage of the ds_jobs is 419965 bytes
After optimization, we achieved a significant reduction in memory usage which is  96.61%


The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies

In [40]:
# Explore the first 5 rows of the new dataset
ds_jobs_transformed.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.919922,Male,True,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,True
1,29725,city_40,0.775879,Male,True,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,False
2,11561,city_21,0.624023,NaN,True,Full time course,Graduate,STEM,5,NaN,NaN,never,83,False
3,33241,city_115,0.789062,NaN,True,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,True
4,666,city_162,0.767090,Male,True,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,False


In [41]:
# Check NaN values of new dataset
ds_jobs_transformed.isna().sum().sort_values(ascending = False)

company_type              6140
company_size              5938
gender                    4508
major_discipline          2813
education_level            460
last_new_job               423
enrolled_university        386
experience                  65
student_id                   0
city                         0
city_development_index       0
relevant_experience          0
training_hours               0
job_change                   0
dtype: int64

Next step is to filter data containing students with 10 or more years of experience at companies with at least 1000 employees. However, the experience and company_size columns have NaN values so we will need to handle the missing values before continuing

In [42]:
ex = (ds_jobs_transformed['experience'].isna().sum()/ds_jobs_transformed.shape[0])*100
co = (ds_jobs_transformed['company_size'].isna().sum()/ds_jobs_transformed.shape[0])*100
print(f'% of missing values in experience column is {ex: .2f}%')
print(f'% of missing values in company_size column is {co: .2f}%')

% of missing values in experience column is  0.34%
% of missing values in company_size column is  30.99%


% of missing values in experience is less than 5% and this number is more than 30% for the company_size column. So we will fillna with 0 for missing values in the experience column and fillna with average value for the company_size column

In [43]:
# Convert the 'experience' column to a numeric type first, then handle NaN values
ds_jobs_transformed['experience'] = pd.to_numeric(ds_jobs_transformed['experience'], errors='coerce').fillna(0)

# Split the company_size string on the dash ('-') to isolate the lower bound and then convert the first part of the split string to an integer
# Handle cases where the company_size might end with a '+' sign
def parse_company_size(size):
    if pd.notna(size):
        if '+' in size:
            return int(size.replace('+', ''))
        elif '-' in size:
            return int(size.split('-')[0])
        else:
            return int(size.replace('<', ''))
            
    return size

ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].apply(parse_company_size)

# Fill missing values in the 'company_size' column with the average value
ds_jobs_transformed['company_size'].fillna(ds_jobs_transformed['company_size'].mean(), inplace=True)

# Check again to make sure 'experience' and 'company_size' don't have any missing values
ex_mi = ds_jobs_transformed['experience'].isna().sum()
co_mi = ds_jobs_transformed['company_size'].isna().sum()

print(f'The \'experience\' column has {ex_mi} missing values')
print(f'The \'company_size\' column has {co_mi} missing values')

The 'experience' column has 0 missing values
The 'company_size' column has 0 missing values


In [44]:
# Filtering with numeric 'experience' and handling 'company_size' appropriately

ds_jobs_transformed_filter = ds_jobs_transformed[

    (ds_jobs_transformed['experience'] >= 10) &

    (ds_jobs_transformed['company_size'] >= 1000)

]

print(len(ds_jobs_transformed_filter))

2476


In [45]:
# The first five rows of the filter dataframe will look like
print(ds_jobs_transformed_filter.head())

    student_id      city  ...  training_hours job_change
5        21651  city_176  ...              24       True
9          699  city_103  ...             123      False
22        6491  city_102  ...              48      False
31       22293  city_103  ...             141      False
34       26494   city_16  ...             145      False

[5 rows x 14 columns]
